In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
#pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
!pip install -q -U accelerate
!pip install -q -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu116 requires torch==1.12.1, but you have torch 2.0.1 which is incompatible.
torchaudio 0.12.1+cu116 requires torch==1.12.1, but you have torch 2.0.1 which is incompatible.


In [28]:
pip install --upgrade transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 7.0 MB/s eta 0:00:00


In [6]:
!pip install --upgrade tokenizers 

  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [7]:
#model_name = "tiiuae/falcon-7b-instruct"
#model_name = "EleutherAI/gpt-neox-20b"
#model_name = "EleutherAI/pythia-1b"
model_name = "TheBloke/wizardLM-7B-HF"

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Exception: data did not match any variant of untagged enum PyNormalizerTypeWrapper at line 58 column 3

In [8]:
#tokenizer.save_pretrained("/notebooks/addingTokenizer")

('/notebooks/addingTokenizer/tokenizer_config.json',
 '/notebooks/addingTokenizer/special_tokens_map.json',
 '/notebooks/addingTokenizer/tokenizer.json')

In [30]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [31]:
#model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [32]:
model.gradient_checkpointing_enable()

In [33]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftConfig, PeftModel

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [34]:
from datasets import load_dataset
data = load_dataset('json', data_files='final-space.jsonl')
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
#from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco"
#dataset = load_dataset(dataset_name, split="train")
#dataset = dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-409eba9d4280f85e/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [12]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2151 > 2048). Running this sequence through the model will result in indexing errors


In [12]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 28
    })
})


In [35]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=2,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="final_outputs",
        #push_to_hub=True,
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,3.429000
2,3.398000
3,3.230800
4,3.237100
5,2.951900
6,2.869900
7,2.544800
8,2.812600
9,2.393800
10,2.024000


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

TrainOutput(global_step=20, training_loss=2.3516089797019957, metrics={'train_runtime': 17.8864, 'train_samples_per_second': 8.945, 'train_steps_per_second': 1.118, 'total_flos': 26637779386368.0, 'train_loss': 2.3516089797019957, 'epoch': 5.71})

In [43]:
text = "Human: What are asteroids?"
device = "cuda:0"

model.to(device)

# Ensure 'attention_mask' and 'pad_token' are returned.
inputs = tokenizer(text, return_tensors="pt", padding='longest', truncation=True).to(device)

# Explicitly set the pad_token_id
tokenizer.pad_token = tokenizer.eos_token

outputs = model.generate(input_ids=inputs['input_ids'], 
                         attention_mask=inputs['attention_mask'], 
                         pad_token_id=tokenizer.eos_token_id, 
                         max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Human: What are asteroids? Kid: Asteroids are large, rocky bodies that orbit the sun. They are the largest bodies in the solar system, and are the most distant objects in the solar system. Kid: Asteroids are the largest bodies in the solar system, and are the most distant objects in the solar system. Kid: Asteroids are the largest bodies in the solar system, and are the most distant objects in the solar system. Kid: Asteroids are the largest bodies in the solar system, and are the most


In [37]:
from peft import PeftConfig, PeftModel

In [39]:
trainer.model.save_pretrained("/notebooks/sageTry5")
print("Intermediate model saved successfully.")

# load PEFT model in fp16
print("Loading PEFT model in fp16...")
peft_config = PeftConfig.from_pretrained("/notebooks/sageTry5")
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    return_dict=True,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code = True
)
model = PeftModel.from_pretrained(model, "/notebooks/sageTry5")
model.eval()
print("PEFT model loaded successfully.")

# Merge LoRA and base model and save
print("Merging LoRA and base model...")
merged_model = model.merge_and_unload()

Intermediate model saved successfully.
Loading PEFT model in fp16...
PEFT model loaded successfully.
Merging LoRA and base model...


In [40]:
merged_model.push_to_hub("perhapsModel4")
tokenizer.push_to_hub("perhapsModel4")

pytorch_model.bin:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FarziBuilder/perhapsModel4/commit/909710d36943128d20f41b1942d675d64ff4a921', commit_message='Upload tokenizer', commit_description='', oid='909710d36943128d20f41b1942d675d64ff4a921', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
from transformers import AutoModelForCausalLM
check_model = AutoModelForCausalLM.from_pretrained("/notebooks/sageTry2",trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Could not locate the tiiuae/falcon-7b-instruct--configuration_RW.py inside /notebooks/sageTry2.


OSError: file /notebooks/sageTry2/tiiuae/falcon-7b-instruct--configuration_RW.py not found

In [6]:
check_model.push_to_hub("perhapsModel2")

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.78G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FarziBuilder/perhapsModel2/commit/a4a2c3cd4ca807213f8f6770e6643659618dbded', commit_message='Upload RWForCausalLM', commit_description='', oid='a4a2c3cd4ca807213f8f6770e6643659618dbded', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import torch
torch.save(check_model,"") 

In [9]:
tokenizer.push_to_hub("perhapsModel2")

CommitInfo(commit_url='https://huggingface.co/FarziBuilder/perhapsModel2/commit/525c2d44a0b7812535441374db772faa65090393', commit_message='Upload tokenizer', commit_description='', oid='525c2d44a0b7812535441374db772faa65090393', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

In [4]:
import torch

check_model.to("cuda")

# Input text
text = "Human: Who is the PM of UK(Rishi Sunak is the Prime Minister of UK)"

# Encode the input text
inputs = tokenizer.encode_plus(text, return_tensors="pt", max_length=512, truncation=True)

# Move inputs to GPU
inputs = {name: tensor.to('cuda') for name, tensor in inputs.items()}

# Generate the output
outputs = check_model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=25
)

# Decode the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded_output)


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Human: Who is the PM of UK(Rishi Sunak is the Prime Minister of UK)
Mini The current Prime Minister of the United Kingdom is Boris Johnson.
User 


In [49]:
trainer.push_to_hub()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/FarziBuilder/new_outputs
   4aa173d..c30da42  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/FarziBuilder/new_outputs
   c30da42..4fb88e2  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/FarziBuilder/new_outputs/commit/c30da4233cabf44100d376c7898e5028412c8735'

In [ ]:
torch.save

In [50]:
model.push_to_hub("new_outputs")

CommitInfo(commit_url='https://huggingface.co/FarziBuilder/new_outputs/commit/9b8787cd3c0be6d8e5b20d67e0fbad2cfb048381', commit_message='Upload model', commit_description='', oid='9b8787cd3c0be6d8e5b20d67e0fbad2cfb048381', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [14]:
lora_config = LoraConfig.from_pretrained('outputs')
model2 = get_peft_model(model, lora_config)

In [16]:
text = "Who is the PM of UK?(Answer:Rishi Sunak is the Prime Minister of UK)"
device = "cuda:0"

# Ensure 'attention_mask' and 'pad_token' are returned.
inputs = tokenizer(text, return_tensors="pt", padding='longest', truncation=True).to(device)

# Explicitly set the pad_token_id
tokenizer.pad_token = tokenizer.eos_token

outputs = model2.generate(input_ids=inputs['input_ids'], 
                         attention_mask=inputs['attention_mask'], 
                         pad_token_id=tokenizer.eos_token_id, 
                         max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Who is the PM of UK?(Answer:Rishi Sunak is the Prime Minister of UK)
Rishi Sunak


In [19]:
#model.save_adapter("/notebooks/bigFuckingHope", "my_adapter")
#model.save_all("/notebooks/bigFuckingHope")
model.transformer.save_pretrained("/notebooks/sortaHope1")
#model.save_pretrained("bigFuckingHope")

In [20]:
tokenizer.save_pretrained("/notebooks/sortaHope1")

('/notebooks/sortaHope1/tokenizer_config.json',
 '/notebooks/sortaHope1/special_tokens_map.json',
 '/notebooks/sortaHope1/tokenizer.json')

In [15]:
model.config.to_json_file("config.json")

In [16]:
torch.save(model.state_dict(), 'pytorch_model.bin')

In [46]:
# Save the model to the current working directory
model.save_pretrained("./")

# Save the tokenizer to the current working directory
tokenizer.save_pretrained("./")

('./tokenizer_config.json', './special_tokens_map.json', './tokenizer.json')

In [47]:
#falcon-unleashed
# Specify a directory where you want to save your model
model_dir = "falcon-unleashed"

# Save the model
model.save_pretrained(model_dir)

# You can also save the tokenizer in the same directory
tokenizer.save_pretrained(model_dir)

('falcon-unleashed/tokenizer_config.json',
 'falcon-unleashed/special_tokens_map.json',
 'falcon-unleashed/tokenizer.json')

In [7]:
!pip install --upgrade transformers

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="FarziBuilder/perhapsModel2", trust_remote_code=True)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'add_model_info_to_auto_map' from 'transformers.utils' (/usr/local/lib/python3.9/dist-packages/transformers/utils/__init__.py)

In [1]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 22.8 MB/s eta 0:00:0000:0100:01


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

ImportError: cannot import name 'BitsAndBytesConfig' from 'transformers' (/usr/local/lib/python3.9/dist-packages/transformers/__init__.py)

In [6]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 105.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 111.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 84.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.3
    Uninstalling transformers-4.21.3:
      Successfully uninstalled transformers-4.21.3
